# How to kick off background runs

This guide covers how to kick off background runs for your agent.
This can be useful for long running jobs.

In [ ]:
# Initialize the client
from langgraph_sdk import get_client

client = get_client()

In [7]:
# List available assistants
assistants = await client.assistants.search()
assistants

[{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
  'graph_id': 'agent',
  'config': {},
  'created_at': '2024-05-18T00:19:39.688822+00:00',
  'updated_at': '2024-05-18T00:19:39.688822+00:00',
  'metadata': {'created_by': 'system'}}]

In [9]:
# Get the first assistant, we will use this one
assistant = assistants[0]
assistant

{'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'graph_id': 'agent',
 'config': {},
 'created_at': '2024-05-18T00:19:39.688822+00:00',
 'updated_at': '2024-05-18T00:19:39.688822+00:00',
 'metadata': {'created_by': 'system'}}

In [10]:
# Create a new thread
thread = await client.threads.create()
thread

{'thread_id': '6ada015b-b47a-4c4f-a5cd-580893cb6d0c',
 'created_at': '2024-05-18T00:50:26.367620+00:00',
 'updated_at': '2024-05-18T00:50:26.367620+00:00',
 'metadata': {}}

In [11]:
# If we list runs on this thread, we can see it is empty
runs = await client.runs.list(thread['thread_id'])
runs

[]

In [ ]:
# Let's kick off a run
input = {"messages": [{"role": "human", "content": "whats the weather in sf"}]}
run = await client.runs.create(thread['thread_id'], assistant["assistant_id"], input=input)


In [13]:
# The first time we poll it, we can see `status=pending`
await client.runs.get(thread['thread_id'], run['run_id'])

{'run_id': 'e843abcb-e478-421b-91e1-a8ae171b14f4',
 'thread_id': '6ada015b-b47a-4c4f-a5cd-580893cb6d0c',
 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'created_at': '2024-05-18T00:50:27.618761+00:00',
 'updated_at': '2024-05-18T00:50:27.618761+00:00',
 'status': 'pending',
 'metadata': {}}

In [14]:
# We can list events for the run
await client.runs.list_events(thread['thread_id'], run['run_id'])

[{'event_id': '3ac6d963-442f-481c-9fde-b8a27bc0e277',
  'run_id': 'e843abcb-e478-421b-91e1-a8ae171b14f4',
  'received_at': '2024-05-18T00:50:29.570649+00:00',
  'span_id': 'd4e4a6ee-da2f-4b5f-b656-1a1f73065161',
  'event': 'on_tool_start',
  'name': 'tavily_search_results_json',
  'data': {'input': {'query': 'weather in san francisco'}},
  'metadata': {'graph_id': 'agent',
   'thread_id': '6ada015b-b47a-4c4f-a5cd-580893cb6d0c',
   'created_by': 'system',
   'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'},
  'tags': ['seq:step:1']},
 {'event_id': 'f8961760-6f13-40e6-9eef-6d4d68e0ed19',
  'run_id': 'e843abcb-e478-421b-91e1-a8ae171b14f4',
  'received_at': '2024-05-18T00:50:29.569708+00:00',
  'span_id': '73295bb1-6cd3-403d-abc1-4f9d96a63894',
  'event': 'on_chain_start',
  'name': 'action',
  'data': {},
  'metadata': {'graph_id': 'agent',
   'thread_id': '6ada015b-b47a-4c4f-a5cd-580893cb6d0c',
   'created_by': 'system',
   'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca'},


In [15]:
# Eventually, it should finish and we should see `status=success`
await client.runs.get(thread['thread_id'], run['run_id'])

{'run_id': 'e843abcb-e478-421b-91e1-a8ae171b14f4',
 'thread_id': '6ada015b-b47a-4c4f-a5cd-580893cb6d0c',
 'assistant_id': 'fe096781-5601-53d2-b2f6-0d3403f7e9ca',
 'created_at': '2024-05-18T00:50:27.618761+00:00',
 'updated_at': '2024-05-18T00:50:27.618761+00:00',
 'status': 'success',
 'metadata': {}}

In [16]:
# We can get the final results
results = await client.runs.list_events(thread['thread_id'], run['run_id'])

In [21]:
# The results are sorted by time, so the most recent (final) step is the 0 index
final_result = results[0]

In [22]:
final_result

{'event_id': '1af2076e-8ec7-4f2e-bc2c-6fbbf586397c',
 'run_id': 'e843abcb-e478-421b-91e1-a8ae171b14f4',
 'received_at': '2024-05-18T00:50:35.557925+00:00',
 'span_id': 'e843abcb-e478-421b-91e1-a8ae171b14f4',
 'event': 'on_chain_end',
 'name': 'LangGraph',
 'data': {'output': {'messages': [{'id': '46b31c3a-01bf-4946-bd5a-fa6a7f6c97ce',
     'name': None,
     'type': 'human',
     'content': 'whats the weather in sf',
     'example': False,
     'additional_kwargs': {},
     'response_metadata': {}},
    {'id': 'run-4885d5a0-cd89-4f00-8558-e85b542a710c',
     'name': None,
     'type': 'ai',
     'content': [{'id': 'toolu_018yyEfJHihdVfWypRNLqDug',
       'name': 'tavily_search_results_json',
       'type': 'tool_use',
       'input': {'query': 'weather in san francisco'}}],
     'example': False,
     'tool_calls': [{'id': 'toolu_018yyEfJHihdVfWypRNLqDug',
       'args': {'query': 'weather in san francisco'},
       'name': 'tavily_search_results_json'}],
     'additional_kwargs': {},


In [28]:
# We can get the content of the final message
final_result['data']['output']['messages'][-1]['content']

"The search results provide the current weather conditions in San Francisco. According to the data, as of 5:45pm on May 17, 2024, the weather in San Francisco is partly cloudy with a temperature of around 64°F (17.8°C). The wind is blowing from the west-northwest at 15 mph (24 km/h) with gusts up to 16 mph (26 km/h). The humidity is 65% and visibility is 9 miles (16 km). The UV index is 5.\n\nSo in summary, it's a partly cloudy spring day in San Francisco with mild temperatures and moderate winds. The weather seems pleasant for being outdoors during the daytime hours."